## GCN-Pytorch

In [ ]:
!git clone https://github.com/dragen1860/GCN-PyTorch.git

Cloning into 'GCN-PyTorch'...
remote: Enumerating objects: 72, done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 72
Unpacking objects: 100% (72/72), done.


In [ ]:
!pwd

/content


In [ ]:
%cd GCN-PyTorch/

/content/GCN-PyTorch


In [ ]:
!python train.py --epochs 400 --dataset pubmed

Namespace(dataset='pubmed', dropout=0.5, early_stopping=10, epochs=400, hidden=16, learning_rate=0.01, max_degree=3, model='gcn', weight_decay=0.0005)
adj: (19717, 19717)
features: (19717, 500)
y: (19717, 3) (19717, 3) (19717, 3)
mask: (19717,) (19717,) (19717,)
x : tensor(indices=tensor([[    0,     0,     0,  ..., 19716, 19716, 19716],
                       [  394,   384,   382,  ...,    16,     7,     1]]),
       values=tensor([0.0283, 0.0162, 0.0125,  ..., 0.0104, 0.0030, 0.0145]),
       device='cuda:0', size=(19717, 500), nnz=988031, layout=torch.sparse_coo)
sp: tensor(indices=tensor([[    0,  1378,  1544,  ..., 19715, 16030, 19716],
                       [    0,     0,     0,  ..., 19715, 19716, 19716]]),
       values=tensor([0.1667, 0.0671, 0.0680,  ..., 0.5000, 0.2500, 0.5000]),
       device='cuda:0', size=(19717, 19717), nnz=108365, layout=torch.sparse_coo)
input dim: 500
output dim: 3
num_features_nonzero: 988031
/content/GCN-PyTorch/utils.py:45: UserWarning: indexing w

## Spektral

In [ ]:
!pip install spektral

     |████████████████████████████████| 122kB 7.0MB/s 


In [ ]:
!git clone https://github.com/danielegrattarola/spektral.git

Cloning into 'spektral'...
remote: Enumerating objects: 7359, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 7359 (delta 164), reused 160 (delta 80), pack-reused 7075
Receiving objects: 100% (7359/7359), 12.26 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (4537/4537), done.


In [ ]:
%cd spektral/examples/node_prediction/

/content/spektral/examples/node_prediction


In [ ]:
!python citation_gcn.py

2021-06-01 09:48:46.923295: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Pre-processing node features
/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2021-06-01 09:48:49.838535: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-01 09:48:49.866615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-01 09:48:49.867240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMe

In [ ]:
4"""
citation_gcn.py
"""
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

learning_rate = 1e-2  # 학습률
seed = 0
epochs = 200          # 데이터를 몇 번 사용해서 학습할 건지
patience = 10
data = "cora"

tf.random.set_seed(seed=seed)  # seed 고정

'''
Load data - datasets/citation.py
dataset name = cora
normalize true = normalize the features
transforms
transforms/layer_preprocess.py & adj_to_sp_tensor.py
LayerPreprocess(GCNConv) - preprocess 기능을 인접행렬에 적용
AdjToSpTensor()] - 인접행렬을 sparse tensor로 변환
'''
dataset = Citation(
    data, normalize_x=True, transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()]
)


# 노드에 대한 avg_loss를 계산하기 위해 binary masks를 sample weights로 변환
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)
    # mask의 0이 아닌 값의 개수


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

# architecture 구현
# n_labels: number of channels in output
# n_input_channels: number of input channels, required for tf 2.1

model = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)
model.compile(
    optimizer=Adam(learning_rate),  # 최적화 알고리즘
    loss=CategoricalCrossentropy(reduction="sum"),  # 손실함수
    weighted_metrics=["acc"], # 학습과 테스트 과정에서 sample_weight이나 class_weight로 가중치를 적용하여 평가할 측정항목의 리스트
)

# Train model
'''
single loader - 단일 그래프를 나타내는 Tensor를 생성
dataset = citation
sample_weights - 주어진 경우 출력에 추가
output -  tuple(inputs, labels) or (inputs, labels, sample_weights)
'''
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))


In [ ]:
# gcn.py
import tensorflow as tf

from spektral.layers.convolutional import gcn_conv


class GCN(tf.keras.Model):
    """
    This model, with its default hyperparameters, implements the architecture
    from the paper:

    > [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907)<br>
    > Thomas N. Kipf and Max Welling

    **Mode**: single, disjoint, mixed, batch.

    **Input**

    - Node features of shape `([batch], n_nodes, n_node_features)`
    - Weighted adjacency matrix of shape `([batch], n_nodes, n_nodes)`

    **Output**

    - Softmax predictions with shape `([batch], n_nodes, n_labels)`.

    **Arguments**

    - `n_labels`: number of channels in output;
    - `channels`: number of channels in first GCNConv layer;
    - `activation`: activation of the first GCNConv layer;
    - `output_activation`: activation of the second GCNConv layer;
    - `use_bias`: whether to add a learnable bias to the two GCNConv layers;
    - `dropout_rate`: `rate` used in `Dropout` layers;
    - `l2_reg`: l2 regularization strength;
    - `n_input_channels`: number of input channels, required for tf 2.1;
    - `**kwargs`: passed to `Model.__init__`.
    """

    def __init__(
        self,
        n_labels,
        channels=16,
        activation="relu",  # 활성화 함수
        output_activation="softmax",
        use_bias=False,
        dropout_rate=0.5,
        l2_reg=2.5e-4,
        n_input_channels=None,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.n_labels = n_labels
        self.channels = channels
        self.activation = activation
        self.output_activation = output_activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.n_input_channels = n_input_channels
        reg = tf.keras.regularizers.l2(l2_reg)
        self._d0 = tf.keras.layers.Dropout(dropout_rate)
        self._gcn0 = gcn_conv.GCNConv(
            channels, activation=activation, kernel_regularizer=reg, use_bias=use_bias
        )
        self._d1 = tf.keras.layers.Dropout(dropout_rate)
        self._gcn1 = gcn_conv.GCNConv(
            n_labels, activation=output_activation, use_bias=use_bias
        )

        if tf.version.VERSION < "2.2":
            if n_input_channels is None:
                raise ValueError("n_input_channels required for tf < 2.2")
            x = tf.keras.Input((n_input_channels,), dtype=tf.float32)
            a = tf.keras.Input((None,), dtype=tf.float32, sparse=True)
            self._set_inputs((x, a))

    def get_config(self):
        return dict(
            n_labels=self.n_labels,
            channels=self.channels,
            activation=self.activation,
            output_activation=self.output_activation,
            use_bias=self.use_bias,
            dropout_rate=self.dropout_rate,
            l2_reg=self.l2_reg,
            n_input_channels=self.n_input_channels,
        )

    def call(self, inputs):
        if len(inputs) == 2:
            x, a = inputs
        else:
            x, a, _ = inputs  # So that the model can be used with DisjointLoader
        if self.n_input_channels is None:
            self.n_input_channels = x.shape[-1]
        else:
            assert self.n_input_channels == x.shape[-1]
        x = self._d0(x)
        x = self._gcn0([x, a])
        x = self._d1(x)
        return self._gcn1([x, a])


In [ ]:
# citation.py

import os
import os.path as osp

import networkx as nx
import numpy as np
import requests
import scipy.sparse as sp
from sklearn.model_selection import train_test_split

from spektral.data import Dataset, Graph
from spektral.datasets.utils import DATASET_FOLDER
from spektral.utils.io import load_binary


class Citation(Dataset):
    """
    The citation datasets Cora, Citeseer and Pubmed.

    Node attributes are bag-of-words vectors representing the most common words
    in the text document associated to each node.
    Two papers are connected if either one cites the other.
    Labels represent the subject area of the paper.

    The train, test, and validation splits are given as binary masks and are
    accessible via the `mask_tr`, `mask_va`, and `mask_te` attributes.

    **Arguments**

    - `name`: name of the dataset to load (`'cora'`, `'citeseer'`, or
    `'pubmed'`);
    - `random_split`: if True, return a randomized split (20 nodes per class
    for training, 30 nodes per class for validation and the remaining nodes for
    testing, as recommended by [Shchur et al. (2018)](https://arxiv.org/abs/1811.05868)).
    If False (default), return the "Planetoid" public splits defined by
    [Yang et al. (2016)](https://arxiv.org/abs/1603.08861).
    - `normalize_x`: if True, normalize the features.
    - `dtype`: numpy dtype of graph data.
    """

    url = "https://github.com/tkipf/gcn/raw/master/gcn/data/{}"
    suffixes = ["x", "y", "tx", "ty", "allx", "ally", "graph", "test.index"]

    def __init__(
        self, name, random_split=False, normalize_x=False, dtype=np.float32, **kwargs
    ):
        if hasattr(dtype, "as_numpy_dtype"):
            # support tf.dtypes
            dtype = dtype.as_numpy_dtype
        if name.lower() not in self.available_datasets():
            raise ValueError(
                "Unknown dataset {}. See {}.available_datasets() for a complete list of"
                "available datasets.".format(name, self.__class__.__name__)
            )
        self.name = name.lower()
        self.random_split = random_split
        self.normalize_x = normalize_x
        self.mask_tr = self.mask_va = self.mask_te = None
        self.dtype = dtype
        super().__init__(**kwargs)

    @property
    def path(self):
        return osp.join(DATASET_FOLDER, "Citation", self.name)

    def read(self):
        objects = [_read_file(self.path, self.name, s) for s in self.suffixes]
        objects = [o.A if sp.issparse(o) else o for o in objects]
        x, y, tx, ty, allx, ally, graph, idx_te = objects

        # Public Planetoid splits. This is the default
        idx_tr = np.arange(y.shape[0])
        idx_va = np.arange(y.shape[0], y.shape[0] + 500)
        idx_te = idx_te.astype(int)
        idx_te_sort = np.sort(idx_te)

        # Fix disconnected nodes in Citeseer
        if self.name == "citeseer":
            idx_te_len = idx_te.max() - idx_te.min() + 1
            tx_ext = np.zeros((idx_te_len, x.shape[1]))
            tx_ext[idx_te_sort - idx_te.min(), :] = tx
            tx = tx_ext
            ty_ext = np.zeros((idx_te_len, y.shape[1]))
            ty_ext[idx_te_sort - idx_te.min(), :] = ty
            ty = ty_ext

        x = np.vstack((allx, tx))
        y = np.vstack((ally, ty))
        x[idx_te, :] = x[idx_te_sort, :]
        y[idx_te, :] = y[idx_te_sort, :]

        # Row-normalize the features
        if self.normalize_x:
            print("Pre-processing node features")
            x = _preprocess_features(x)

        if self.random_split:
            # Throw away public splits and compute random ones like Shchur et al.
            indices = np.arange(y.shape[0])
            n_classes = y.shape[1]
            idx_tr, idx_te, _, y_te = train_test_split(
                indices, y, train_size=20 * n_classes, stratify=y
            )
            idx_va, idx_te = train_test_split(
                idx_te, train_size=30 * n_classes, stratify=y_te
            )

        # Adjacency matrix
        a = nx.adjacency_matrix(nx.from_dict_of_lists(graph))  # CSR
        a.setdiag(0)
        a.eliminate_zeros()

        # Train/valid/test masks
        self.mask_tr = _idx_to_mask(idx_tr, y.shape[0])
        self.mask_va = _idx_to_mask(idx_va, y.shape[0])
        self.mask_te = _idx_to_mask(idx_te, y.shape[0])

        return [
            Graph(
                x=x.astype(self.dtype),
                a=a.astype(self.dtype),
                y=y.astype(self.dtype),
            )
        ]

    def download(self):
        print("Downloading {} dataset.".format(self.name))
        os.makedirs(self.path, exist_ok=True)
        for n in self.suffixes:
            f_name = "ind.{}.{}".format(self.name, n)
            req = requests.get(self.url.format(f_name))
            if req.status_code == 404:
                raise ValueError(
                    "Cannot download dataset ({} returned 404).".format(
                        self.url.format(f_name)
                    )
                )
            with open(os.path.join(self.path, f_name), "wb") as out_file:
                out_file.write(req.content)

    @staticmethod
    def available_datasets():
        return ["cora", "citeseer", "pubmed"]


class Cora(Citation):
    """
    Alias for `Citation('cora')`.
    """

    def __init__(self, random_split=False, normalize_x=False, **kwargs):
        super().__init__(
            "cora", random_split=random_split, normalize_x=normalize_x, **kwargs
        )


class Citeseer(Citation):
    """
    Alias for `Citation('citeseer')`.
    """

    def __init__(self, random_split=False, normalize_x=False, **kwargs):
        super().__init__(
            "citeseer", random_split=random_split, normalize_x=normalize_x, **kwargs
        )


class Pubmed(Citation):
    """
    Alias for `Citation('pubmed')`.
    """

    def __init__(self, random_split=False, normalize_x=False, **kwargs):
        super().__init__(
            "pubmed", random_split=random_split, normalize_x=normalize_x, **kwargs
        )


def _read_file(path, name, suffix):
    full_fname = os.path.join(path, "ind.{}.{}".format(name, suffix))
    if suffix == "test.index":
        return np.loadtxt(full_fname)

    return load_binary(full_fname)


def _idx_to_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def _preprocess_features(features):
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.0
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features
